Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# MySQL Relational Database


<img src="datamodel.png" alt="Data Model" style="width: 300px; float: right; margin-left: 20px; border: 1px solid">

This sample database has the hospitals, systems, services, and visits information we talked about in the slides.  The model (table names, columns, and relationships) is show to the right.

In this model, there are four tables:
* System Affiliations
* Hospitals
* ED Visits
* Hospital Services

In these examples, we're going to ask a few questions and answer them with both SQL and Python Pandas data processes.


## Setup

Create a connection to the MySQL database

In [1]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2019.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2019'
password = 'SLUcor2019'
database = 'slucor2019'

conn = create_engine('mysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + ':' + 
                     str(port) + '/' + 
                     database)

## Give me a list of hospitals

In [2]:
pd.read_sql_query("SELECT * FROM hospitals", conn)

,hospital_name,city,system_name,beds
0,BJH,St. Louis,BJC,1243
1,Mercy STL,Ladue,Mercy,1120
2,MoBap,Ladue,BJC,443


# All the hospitals that are not affiliated with the Catholic church

Using SQL, we join the hospitals with system_affiliations tables to get a list of hospitals.

In [ ]:
pd.read_sql_query("""
  SELECT h.*, s.affiliation
  FROM 
    hospitals h JOIN
    system_affiliations s ON h.system_name = s.system_name
  WHERE
    affiliation != 'Catholic'
""", conn)

## Read in all the DB tables into an array of Dataframes

1. Get a list of all of the tables
2. Read each table into a dataframe; and
3. Store that in a dictionary of dataframes

In [ ]:
tables = {}

for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = t.item()
    print(name)
    tables[name] = pd.read_sql("SELECT * FROM "+name,conn)

# All the hospitals that are not affiliated with the Catholic church

This time, using Pandas

In [ ]:
hospitals = tables['hospitals'].merge(tables['system_affiliations'])

In [ ]:
hospitals[hospitals['affiliation'] != 'Catholic']

## Hospitals with more than 43,000 ED visits each year

First in SQL

In [ ]:
pd.read_sql_query("""
  SELECT h.*, v.*
  FROM 
    hospitals h JOIN
    ed_visits v ON h.hospital_name = v.hospital_name
  WHERE
    v.ed_visits > 43000
""", conn)

Now, let's use SQL to pivot that so that we can see 2016 and 2017 in two separate columns...

# ...

Come on.  Let's do it?

# ...

Oh.  That isn't supported.  :(

## Let's try with Pandas

In [ ]:
visits = tables['hospitals'].merge(tables['ed_visits'])

In [ ]:
visits

In [ ]:
visits = visits[visits['ed_visits'] > 43000]

In [ ]:
pd.pivot_table(visits, 
               index=['hospital_name','city','system_name','beds'], 
               columns='year', 
               values='ed_visits').reset_index()

## OK.  We can do it in SQL...

But can you figure out why this is a bad idea?

In [ ]:
pd.read_sql_query("""
  SELECT 
    h.*, 
    MIN(CASE WHEN v.year = 2016 THEN v.ed_visits ELSE NULL END) as visits_2016,
    MIN(CASE WHEN v.year = 2017 THEN v.ed_visits ELSE NULL END) as visits_2017
  FROM 
    hospitals h JOIN
    ed_visits v ON h.hospital_name = v.hospital_name
  WHERE
    v.ed_visits > 43000
  GROUP BY 
    h.hospital_name, h.city, h.system_name, h.beds
""", conn)

## Now, let's create some new data from scratch

1. Create a dataframe
2. Use a name that includes your username so that it's unique
3. Write the df to a new table
4. Query your data back out

See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html

In [ ]:
columns = ['hospital_name','year','ip_visits']
data = [
    ['BJH',2016,3124],
    ['Mercy STL',2016,4321],
    ['MoBap',2016,2783]
]
df = pd.DataFrame(data, columns=columns)

In [ ]:
df

In [ ]:
import getpass
myname = getpass.getuser()
myname

In [ ]:
df.to_sql(myname + '_ip_visits', conn)

In [ ]:
ip_visits = pd.read_sql_query('SELECT * FROM ' + myname + '_ip_visits', conn)

In [ ]:
ip_visits

In [ ]:
pd.read_sql_query('SHOW TABLES', conn)

## And we need to clean up after ourselves by dropping our table

In [ ]:
with conn.connect() as c:
    c.execute('DROP TABLE IF EXISTS ' + myname + '_ip_visits')